In [94]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from scipy import sparse 
import sys
from gaussiandata import SyntheticTarget
from online_prototypes import greedy_select_protos_online
from online_criticism import greedy_select_criticism_online
#import greedy_functions
import math
import scipy

In [95]:
SEED = 666
number_of_gaussians = 2
ex = SyntheticTarget(seed=SEED, dist_components=number_of_gaussians)

N = 1000
M = 10
data = ex.draw_sample(N)
print  data.size
print data[(2,3,4),:]
gamma = 2
kernel_matrix = rbf_kernel(data, gamma = gamma)

2000
[[ 2.58801945 -4.10201855]
 [-1.1745815  -3.81182871]
 [ 3.41529632 -3.59914177]]


[[-1.67010453 -4.59664315]
 [ 3.34775394 -3.18054631]]
[1, 2]


0.42457731180256464

In [98]:
def rbf(x,y, gamma):
    x = np.asarray(x)
    y = np.asarray(y)
    return scipy.exp(-gamma*np.linalg.norm(x-y))

def our_rbf_kernel(current_kernel_subset,  gamma, x_new, n, M, data, current_indices):
    new_row = np.zeros(n+1)
    new_col = np.zeros(M+1)
    for i in range(n+1):
        new_row[i] = rbf(x = x_new, y = data[i], gamma = gamma)
    print new_row.shape
    new_col = new_row[current_indices]
    print new_col.shape
    new_kernel_subset = np.hstack((current_kernel_subset, new_col.transpose()))
    new_kernel_subset = np.vstack((new_kernel_subset, new_row))
    
    return new_kernel_subset

In [ ]:
our_rbf_kernel(kernel_matrix)

In [99]:
gamma = 2
prototype_indices = np.arange(M) #assign first M points as prototypes. CONSIDER another method later.
criticism_indices = np.arange(M, 2*M) # assign second M points as prototypes.

#calculate the initial kernel matrix for the first M points
#current_kernel_matrix_proto = rbf_kernel(data[prototype_indices,:], gamma=gamma)
#print current_kernel_matrix_proto.shape

#current_kernel_matrix_criticism = rbf_kernel(data[criticism_indices,:], gamma = gamma) #to check with Petya

current_kernel_matrix = rbf_kernel(data[range(2*M)], gamma = gamma)
current_kernel_matrix_proto = current_kernel_matrix[:M]
current_kernel_matrix_criticism = current_kernel_matrix[range(M, 2*M)]
print current_kernel_matrix_proto.shape
print current_kernel_matrix_criticism.shape

for i in range(2*M, N):
    new_point_index = i # assign the new data point index
    
    accept = 1 # decide whether to skip the criticism part or not 
    
    #prototype part
    if accept == 1:
        #add new column and row to the kernel matrix for the new point; 
        #kernel matrix is always growing
        #current_kernel_subset,  gamma, x_new, n, timeseries):
        print current_kernel_matrix_proto.shape
        new_kernel_matrix_proto = our_rbf_kernel(current_kernel_subset = current_kernel_matrix_proto, gamma = gamma,  
                                                 x_new= data[i], n = i, M = M,  data = data[range(i+1)], 
                                                 current_indices = prototype_indices)
        
        print new_kernel_matrix_proto.shape
        #run the greedy algorithm on these M+1 data points to select M prototypes
        prototype_indices_temp = greedy_select_protos_online(new_kernel_matrix_proto, 
                                                  selectedprotos = np.vstack((prototype_indices, i)), m = M)
    
        if (prototype_indices_temp == prototype_indices).all():
            accept == 0
        
        else:
            prototype_indices = prototype_indices_temp # update the indices
            current_kernel_matrix_proto = new_kernel_matrix_proto # update the kernel matrix
    
    
    #criticism part
    if accept == 0:
        
        new_kernel_matrix_criticism = our_rbf_kernel(current_kernel_subset = current_kernel_matrix_criticism, 
                                                     gamma = gamma,  x_new= data[i], n = i, M = M, 
                                                     data = data[range(i+1)], current_indices = criticism_indices)
        
        #run the greedy algorithm on these M+1 data points to select M criticism
        criticism_indices_temp = greedy_select_criticism_online(new_kernel_matrix_criticism, 
                                                  selectedprotos = prototype_indices, selectedcritic = criticism_indices, c = M, reg='logdet')

        
        if (criticism_indices_temp == criticism_indices).all():
            accept == 0
        
        else:
            criticism_indices = criticism_indices_temp # update the indices
            current_kernel_matrix_criticism = new_kernel_matrix_criticism # update the kernel matrix
    
    

(10, 20)
(10, 20)
(10, 20)
(21,)
(10,)


ValueError: all the input arrays must have same number of dimensions

In [ ]:
np.arange(M+1)

In [ ]:
current_kernel_subset =  np.random.randn(5,10)
our_rbf_kernel(current_kernel_subset = current_kernel_subset, gamma = 2, x_new = 6, n = 10, timeseries = np.arange(10))


In [ ]:
new_row = np.zeros(10)

In [ ]:
print new_row

In [1]:
range(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
A =  np.random.randn(5,10)
B =  np.random.randn(5,10)


In [72]:
prototype_indices =range(M) 
prototype_indices

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]